# Data Update (Run For Each GW)

This notebook updates the training data weekly for the gameweek points prediction model. The notebook takes in new PL stats per gameweek and adds them as new rows to the original training dataset.

### REMINDERS BEFORE RUNNING:
- Download updated player_stats_2223 (https://github.com/vaastav/Fantasy-Premier-League/tree/master/data/2022-23)
- Download updated team_standard_stats_2223 & player_standard_stats_2223 (https://fbref.com/en/comps/9/stats/Premier-League-Stats)

In [303]:
#Import relevant libraries and packages
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#Paths
path = Path('Data')
path_20_21 = Path('Data/2020-21')
path_21_22 = Path('Data/2021-22')
path_22_23 = Path('Data/2022-23')

#Import data sets
#Training data
training_data = pd.read_csv(path/'training_data_updated.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})
training_data = training_data.reset_index()
# original_training_data = pd.read_csv(path/'training_data.csv', index_col=0, 
#                        dtype={'season':str,
#                               'comp':str,
#                               'squad':str})

#22-23 season gameweeks
season_gws = pd.read_csv(path/'remaining_season.csv', index_col=0)

#Player info 21-22 and 22-23
player_stats_2122 = pd.read_csv(path_21_22/'gws/merged_gw.csv')
player_stats_2223 = pd.read_csv(path_22_23/'gws/merged_gw.csv')

#Team stats 20-21, 21-22 and 22-23
team_standard_stats_2021 = pd.read_csv(path_20_21/'team_standard_stats_2021.csv')
team_standard_stats_2122 = pd.read_csv(path_21_22/'team_standard_stats_2122.csv')
team_standard_stats_2223 = pd.read_csv(path_22_23/'team_standard_stats_2223.csv')

#Player stats 21-22 and 22-23
player_standard_stats_2122 = pd.read_csv(path_21_22/'player_standard_stats_2122.csv')
player_standard_stats_2223 = pd.read_csv(path_22_23/'player_standard_stats_2223.csv')

#Player's raw data 22-23
players_raw = pd.read_csv(path_22_23/'players_raw.csv')

#22-23 Teams (Teams & IDs)
teams = pd.read_csv(path_22_23/'teams.csv')

#Cleaned player data 22-23
cleaned_players = pd.read_csv(path_22_23/'cleaned_players.csv')

In [304]:
#Function to merge dfs by partial match (substring)
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

We will create a new training data frame with the original training data from the 20-21 season, the 21-22 season (adding gw 38 data), and the 22-23 season (adding each new gw weekly):

## 20-21 Training Data:

In [123]:
# original_training_data_2021 = original_training_data[original_training_data['season'] == '2021'][['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
#        'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
#        'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
#        'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
#        'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]

# original_training_data_2021['team'] = original_training_data_2021['team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# original_training_data_2021['opponent_team'] = original_training_data_2021['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# training_data_2021 = original_training_data_2021.merge(team_standard_stats_2021, on='team')
# training_data_2021 = training_data_2021.sort_values(['gw','team'])
# training_data_2021 = training_data_2021.reset_index()
# training_data_2021 = training_data_2021.drop('index', axis=1)
# training_data_2021

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,2,1,Arsenal,Fulham,False,2021,90,6,0,...,4.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
1,Alexandre Lacazette,4,1,Arsenal,Fulham,False,2021,86,7,0,...,48.0,0,3,0,0.4,0.2,0.4,1.41,0.96,1.29
2,Bernd Leno,1,1,Arsenal,Fulham,False,2021,90,7,0,...,0.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
3,Bukayo Saka,3,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
4,Calum Chambers,2,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24359,Nélson Cabral Semedo,2,38,Wolves,Manchester Utd,True,2021,90,8,0,...,19.0,0,2,1,0.1,0.1,0.1,1.05,0.70,0.97
24360,Conor Coady,2,38,Wolves,Manchester Utd,True,2021,90,1,0,...,0.0,0,2,1,0.0,0.0,0.0,1.05,0.70,0.97
24361,Theo Corbeanu,3,38,Wolves,Manchester Utd,True,2021,0,0,0,...,0.0,0,2,1,NaN,NaN,NaN,1.05,0.70,0.97
24362,Matija Sarkic,1,38,Wolves,Manchester Utd,True,2021,0,0,0,...,0.0,0,2,1,NaN,NaN,NaN,1.05,0.70,0.97


## 21-22 Training Data:

In [124]:
# original_training_data_2122 = original_training_data[original_training_data['season'] == '2122'][['player', 'position', 
#        'gw', 'team', 'opponent_team', 'was_home',
#        'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
#        'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
#        'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
#        'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]
                                                                                                 
# original_training_data_2122['team'] = original_training_data_2122['team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# original_training_data_2122['opponent_team'] = original_training_data_2122['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
#                                           'Newcastle United': 'Newcastle Utd',
#                                           'West Ham United': 'West Ham',
#                                           'Tottenham Hotspur': 'Tottenham',
#                                           'Brighton and Hove Albion': 'Brighton',
#                                           'Wolverhampton Wanderers': 'Wolves',
#                                           'Sheffield United': 'Sheffield Utd',
#                                           'West Bromwich Albion': 'West Brom'})

# training_data_2122 = original_training_data_2122.merge(team_standard_stats_2122, on='team')
# training_data_2122 = training_data_2122.sort_values(['gw','team'])
# training_data_2122 = training_data_2122.reset_index()
# training_data_2122 = training_data_2122.drop('index', axis=1)
# training_data_2122

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,3,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
1,Albert Sambi Lokonga,3,1,Arsenal,Brentford,False,2122,90,2,0,...,8.0,0,0,2,0.0,0.1,0.0,1.58,0.97,1.42
2,Alexandre Lacazette,4,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
3,Ben White,2,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
4,Bernd Leno,1,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24705,Ryan Giles,3,37,Wolves,Norwich,True,2122,0,0,0,...,0.0,0,1,1,NaN,NaN,NaN,0.95,0.65,0.93
24706,Rúben Diogo da Silva Neves,3,37,Wolves,Norwich,True,2122,90,2,0,...,8.0,0,1,1,0.1,0.0,0.1,0.95,0.65,0.93
24707,Toti António Gomes,2,37,Wolves,Norwich,True,2122,90,2,0,...,28.0,0,1,1,0.1,0.0,0.1,0.95,0.65,0.93
24708,Willy Boly,2,37,Wolves,Norwich,True,2122,90,2,0,...,2.0,0,1,1,0.0,0.0,0.0,0.95,0.65,0.93


#### Add Gameweek 38 data:

In [125]:
# #Last season's gameweek 38 data (RUN JUST ONCE) 
# player_stats_2122 = player_stats_2122[player_stats_2122['GW'] == 38]
# player_stats_2122['season'] = '2122'
# relevant_columns = ['name', 'position', 'GW', 'team', 'opponent_team', 'was_home', 'season', 'minutes','total_points',
#                     'assists', 'bonus', 'bps', 'clean_sheets',
#                     'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
#                     'influence', 'penalties_saved', 'red_cards', 'saves', 'threat',
#                     'yellow_cards', 'team_a_score', 'team_h_score']
# player_stats_2122 = player_stats_2122[relevant_columns]
# player_stats_2122 = player_stats_2122.rename(columns={'name': 'player', 'GW':'gw'})

# #Change position from string to int
# def position_assignment(data):
#     if data['position'] == 'GK':
#         return 1
#     if data['position'] == 'DEF':
#         return 2
#     if data['position'] == 'MID':
#         return 3
#     if data['position'] == 'FWD':
#         return 4
    
# player_stats_2122['position'] = player_stats_2122.apply(position_assignment, axis = 1)

# #Change opponent_team from int to string
# def team_assignment(data):
#     if data['opponent_team'] == 1:
#         return 'Arsenal'
#     if data['opponent_team'] == 2:
#         return 'Aston Villa'
#     if data['opponent_team'] == 3:
#         return 'Brentford'
#     if data['opponent_team'] == 4:
#         return 'Brighton'
#     if data['opponent_team'] == 5:
#         return 'Burnley'
#     if data['opponent_team'] == 6:
#         return 'Chelsea'
#     if data['opponent_team'] == 7:
#         return 'Crystal Palace'
#     if data['opponent_team'] == 8:
#         return 'Everton'
#     if data['opponent_team'] == 9:
#         return 'Leicester City'
#     if data['opponent_team'] == 10:
#         return 'Leeds'
#     if data['opponent_team'] == 11:
#         return 'Liverpool'
#     if data['opponent_team'] == 12:
#         return 'Manchester City'
#     if data['opponent_team'] == 13:
#         return 'Manchester Utd'
#     if data['opponent_team'] == 14:
#         return 'Newcastle Utd'
#     if data['opponent_team'] == 15:
#         return 'Norwich'
#     if data['opponent_team'] == 16:
#         return 'Southampton'
#     if data['opponent_team'] == 17:
#         return 'Tottenham'
#     if data['opponent_team'] == 18:
#         return 'Watford'
#     if data['opponent_team'] == 19:
#         return 'West Ham'
#     if data['opponent_team'] == 20:
#         return 'Wolves'
    
# player_stats_2122['opponent_team'] = player_stats_2122.apply(team_assignment, axis = 1)
# gw_38 = player_stats_2122
# gw_38 = gw_38.sort_values(['team'])
# gw_38

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,goals_scored,ict_index,influence,penalties_saved,red_cards,saves,threat,yellow_cards,team_a_score,team_h_score
25122,Emile Smith Rowe,3,38,Arsenal,Everton,True,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,5
25033,Pablo Marí,2,38,Arsenal,Everton,True,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,5
25059,Mazeed Ogungbo,2,38,Arsenal,Everton,True,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,5
25110,Pierre-Emerick Aubameyang,4,38,Arsenal,Everton,True,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,5
24849,Cédric Soares,2,38,Arsenal,Everton,True,2122,90,14,1,...,1,0.0,0.0,0,0,0,0.0,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25111,José Malheiro de Sá,1,38,Wolves,Liverpool,False,2122,45,1,0,...,0,0.0,0.0,0,0,1,0.0,0,1,3
24774,Luke Cundle,3,38,Wolves,Liverpool,False,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,3
24931,Francisco Machado Mota de Castro Trincão,3,38,Wolves,Liverpool,False,2122,1,1,0,...,0,0.0,0.0,0,0,0,0.0,0,1,3
25195,Hugo Bueno,2,38,Wolves,Liverpool,False,2122,0,0,0,...,0,0.0,0.0,0,0,0,0.0,0,1,3


In [126]:
# training_data_2122 = pd.concat([training_data_2122, gw_38])
# training_data_2122

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,3,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
1,Albert Sambi Lokonga,3,1,Arsenal,Brentford,False,2122,90,2,0,...,8.0,0,0,2,0.0,0.1,0.0,1.58,0.97,1.42
2,Alexandre Lacazette,4,1,Arsenal,Brentford,False,2122,0,0,0,...,0.0,0,0,2,NaN,NaN,NaN,1.58,0.97,1.42
3,Ben White,2,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
4,Bernd Leno,1,1,Arsenal,Brentford,False,2122,90,1,0,...,0.0,0,0,2,0.0,0.0,0.0,1.58,0.97,1.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25111,José Malheiro de Sá,1,38,Wolves,Liverpool,False,2122,45,1,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
24774,Luke Cundle,3,38,Wolves,Liverpool,False,2122,0,0,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
24931,Francisco Machado Mota de Castro Trincão,3,38,Wolves,Liverpool,False,2122,1,1,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN
25195,Hugo Bueno,2,38,Wolves,Liverpool,False,2122,0,0,0,...,0.0,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# #Concatenate 20-21 training data to 21-22 training data
# updated_training_data = pd.concat([training_data_2021, training_data_2122])

## Gameweek 22-23 Data:
#### UPDATE GAMEWEEK EVERY WEEK

In [305]:
#REMINDER TO CHANGE GW WEEKLY TO MOST RECENT GW
gameweek = 3

#Player stats for most recent gameweek
player_stats = player_stats_2223[player_stats_2223['GW'] == gameweek]
relevant_columns = ['name', 'minutes','total_points', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'penalties_saved', 'red_cards', 'saves', 'threat',
       'yellow_cards', 'team_a_score', 'team_h_score']
player_stats = player_stats[relevant_columns]
player_stats = player_stats.rename(columns={'name': 'player'})
player_stats

,player,minutes,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,penalties_saved,red_cards,saves,threat,yellow_cards,team_a_score,team_h_score
1154,Nathan Redmond,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,2,1
1155,Junior Stanislas,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,3,0
1156,Armando Broja,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,0,3
1157,Fabian Schär,90,0,0,0,9,0,0.2,3,0,2.5,24.4,0,0,0,0.0,1,3,3
1158,Jonny Evans,90,1,0,0,14,0,1.3,2,0,1.9,18.0,0,0,0,0.0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,Oliver Skipp,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,0,1
1742,Ryan Sessegnon,14,1,0,0,4,0,0.1,0,0,0.2,0.2,0,0,0,2.0,0,0,1
1743,Ashley Young,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,1,3
1744,Jeremy Sarmiento Morante,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0.0,0,2,0


In [306]:
#Player Raw Data Merged with player's team
players_raw = players_raw[['first_name', 'second_name', 'team_code']]
teams = teams[['code', 'name']]
players_raw = players_raw.merge(teams, left_on = 'team_code', right_on= 'code')
players_raw['player'] = players_raw['first_name'] + ' ' + players_raw['second_name']
players_raw = players_raw.rename(columns={'name': 'team'})
players_raw = players_raw[['player', 'team']]
players_raw['team'] = players_raw['team'].replace({'Man Utd': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves',
                                          'Leicester': 'Leicester City',
                                          'Man City': 'Manchester City',
                                          'Newcastle': 'Newcastle Utd',
                                          "Nott'm Forest": 'Nottingham Forest',
                                          'Spurs': 'Tottenham'})
players_raw

,player,team
0,Cédric Alves Soares,Arsenal
1,Granit Xhaka,Arsenal
2,Mohamed Elneny,Arsenal
3,Rob Holding,Arsenal
4,Thomas Partey,Arsenal
...,...,...
587,Joseph Hodge,Wolves
588,Connor Ronan,Wolves
589,Chem Campbell,Wolves
590,Gonçalo Manuel Ganchinho Guedes,Wolves


#### UPDATE GAMEWEEK HERE:

In [307]:
#Gameweek Data
season_gws['opponent_team'] = season_gws['opponent_team'].replace({'Manchester United': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves'})

season_gws['team'] = season_gws['team'].replace({'Manchester United': 'Manchester Utd', 
                                          'Newcastle United': 'Newcastle Utd',
                                          'West Ham United': 'West Ham',
                                          'Tottenham Hotspur': 'Tottenham',
                                          'Brighton and Hove Albion': 'Brighton',
                                          'Wolverhampton Wanderers': 'Wolves'})

season_gws = season_gws[['gw','team', 'opponent_team', 'was_home', 'season']]
season_gws = season_gws.drop_duplicates()
season_gws = season_gws.reset_index().drop('index', axis=1)

#CHANGE GAMEWEEK HERE
season_gws = season_gws[season_gws['gw'] == 3]
season_gws

,gw,team,opponent_team,was_home,season
20,3,Leeds,Chelsea,True,2223
21,3,Manchester Utd,Liverpool,True,2223
22,3,Newcastle Utd,Manchester City,True,2223
23,3,West Ham,Brighton,True,2223
24,3,Bournemouth,Arsenal,True,2223
...,...,...,...,...,...
405,3,Aston Villa,Crystal Palace,False,2223
406,3,Brentford,Fulham,False,2223
407,3,Nottingham Forest,Everton,False,2223
408,3,Wolves,Tottenham,False,2223


In [308]:
#Merge gameweek info with player names
season_player_merge = season_gws.merge(players_raw, on='team')
season_player_merge = season_player_merge[['player', 'gw', 'team', 'opponent_team', 'was_home', 'season']]
season_player_merge

,player,gw,team,opponent_team,was_home,season
0,Liam Cooper,3,Leeds,Chelsea,True,2223
1,Luke Ayling,3,Leeds,Chelsea,True,2223
2,Mateusz Klich,3,Leeds,Chelsea,True,2223
3,Adam Forshaw,3,Leeds,Chelsea,True,2223
4,Rodrigo Moreno,3,Leeds,Chelsea,True,2223
...,...,...,...,...,...,...
587,Gavin Bazunu,3,Southampton,Leicester City,False,2223
588,Armel Bella-Kotchap,3,Southampton,Leicester City,False,2223
589,Willy Caballero,3,Southampton,Leicester City,False,2223
590,Joe Ayodele-Aribo,3,Southampton,Leicester City,False,2223


In [309]:
#Add player's position
cleaned_players['player'] = cleaned_players['first_name'] + ' ' + cleaned_players['second_name']
cleaned_players = cleaned_players[['player', 'element_type']]
cleaned_players = cleaned_players.rename(columns={'element_type': 'position'})
season_player_merge = season_player_merge.merge(cleaned_players, on='player')

#Convert position from string to int
def position_assignment(data):
    if data['position'] == 'GK':
        return 1
    if data['position'] == 'DEF':
        return 2
    if data['position'] == 'MID':
        return 3
    if data['position'] == 'FWD':
        return 4

season_player_merge['position'] = season_player_merge.apply(position_assignment, axis = 1)

#Ordered and clean df with player gw data
season_player_merge = season_player_merge[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home', 'season']]
season_player_merge = season_player_merge.drop_duplicates()
season_player_merge

,player,gw,team,opponent_team,was_home,season,position
0,Liam Cooper,3,Leeds,Chelsea,True,2223,DEF
1,Luke Ayling,3,Leeds,Chelsea,True,2223,DEF
2,Mateusz Klich,3,Leeds,Chelsea,True,2223,MID
3,Adam Forshaw,3,Leeds,Chelsea,True,2223,MID
4,Rodrigo Moreno,3,Leeds,Chelsea,True,2223,MID
...,...,...,...,...,...,...,...
589,Gavin Bazunu,3,Southampton,Leicester City,False,2223,GK
590,Armel Bella-Kotchap,3,Southampton,Leicester City,False,2223,DEF
591,Willy Caballero,3,Southampton,Leicester City,False,2223,GK
592,Joe Ayodele-Aribo,3,Southampton,Leicester City,False,2223,MID


In [311]:
#Merging last gameweek's player stats with last gameweek's player info and adding players' xg, xa, and npxg
season_gw = fuzzy_merge(season_player_merge, player_stats, 'player', 'player', threshold=91)
season_gw_stats = season_gw.merge(player_stats, left_on = 'matches', right_on = 'player')
season_gw_stats = season_gw_stats.drop(['player_x', 'matches'], axis=1)
season_gw_stats = season_gw_stats.rename(columns={'player_y': 'player'})
season_gw_stats = season_gw_stats[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score']]

season_gw_stats = fuzzy_merge(season_gw_stats, player_standard_stats_2223, 'player', 'player', threshold=91)
season_gw_stats['matches'].replace('', np.nan, inplace=True)
season_gw_no_match = season_gw_stats[season_gw_stats['matches'].isna()]
season_gw_no_match[['xg', 'xa', 'npxg']] = 0
season_gw_no_match = season_gw_no_match.drop('matches', axis=1)
season_gw_stats = season_gw_stats.dropna(subset=['matches'])
season_gw_stats = season_gw_stats.merge(player_standard_stats_2223, left_on = 'matches', right_on = 'player')
season_gw_stats = season_gw_stats.drop(['player_x', 'matches'], axis=1)
season_gw_stats = season_gw_stats.rename(columns={'player_y': 'player'})
season_gw_stats = season_gw_stats[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg']]
season_gw_stats = pd.concat([season_gw_stats, season_gw_no_match])
season_gw_stats

/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_24679/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_gw_no_match[['xg', 'xa', 'npxg']] = 0
/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_24679/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_gw_no_match[['xg', 'xa', 'npxg']] = 0
/var/folders/mh/djnbz20x6yx5_43kjtkrmxmh0000gn/T/ipykernel_24679/868439791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,penalties_saved,red_cards,saves,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg
0,Mateusz Klich,3,3,Leeds,Chelsea,True,2223,7,1,0,...,0,0,0,0.0,0,0,3,0.00,0.25,0.00
1,Adam Forshaw,3,3,Leeds,Chelsea,True,2223,29,1,0,...,0,0,0,0.0,0,0,3,0.00,0.00,0.00
2,Patrick Bamford,4,3,Leeds,Chelsea,True,2223,0,0,0,...,0,0,0,0.0,0,0,3,0.37,0.00,0.37
3,Diego Llorente,2,3,Leeds,Chelsea,True,2223,90,8,0,...,0,0,0,4.0,0,0,3,0.03,0.00,0.03
4,Robin Koch,2,3,Leeds,Chelsea,True,2223,90,6,0,...,0,0,0,0.0,0,0,3,0.01,0.00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,Will Smallbone,3,3,Southampton,Leicester City,False,2223,0,0,0,...,0,0,0,0.0,0,2,1,0.00,0.00,0.00
582,Tino Livramento,2,3,Southampton,Leicester City,False,2223,0,0,0,...,0,0,0,0.0,0,2,1,0.00,0.00,0.00
584,Mateusz Lis,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0,0,0,0.0,0,2,1,0.00,0.00,0.00
587,Willy Caballero,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0,0,0,0.0,0,2,1,0.00,0.00,0.00


#### UPDATE GAMEWEEK HERE:

In [312]:
#Add team xg, xa, npxg
gw4 = season_gw_stats 
gw4 = gw4.merge(team_standard_stats_2223, left_on = 'team', right_on ='team')
gw4 = gw4[['player', 'position', 'gw', 'team', 'opponent_team', 'was_home',
       'season', 'minutes', 'total_points', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'penalties_saved', 'red_cards', 'saves',
       'threat', 'yellow_cards', 'team_a_score', 'team_h_score', 'xg', 'xa', 'npxg',
       'team_xg', 'team_xa', 'team_npxg']]

gw4

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Mateusz Klich,3,3,Leeds,Chelsea,True,2223,7,1,0,...,0.0,0,0,3,0.00,0.25,0.00,1.52,0.72,1.52
1,Adam Forshaw,3,3,Leeds,Chelsea,True,2223,29,1,0,...,0.0,0,0,3,0.00,0.00,0.00,1.52,0.72,1.52
2,Patrick Bamford,4,3,Leeds,Chelsea,True,2223,0,0,0,...,0.0,0,0,3,0.37,0.00,0.37,1.52,0.72,1.52
3,Diego Llorente,2,3,Leeds,Chelsea,True,2223,90,8,0,...,4.0,0,0,3,0.03,0.00,0.03,1.52,0.72,1.52
4,Robin Koch,2,3,Leeds,Chelsea,True,2223,90,6,0,...,0.0,0,0,3,0.01,0.00,0.01,1.52,0.72,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,Will Smallbone,3,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.00,0.00,0.00,0.76,0.62,0.76
585,Tino Livramento,2,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.00,0.00,0.00,0.76,0.62,0.76
586,Mateusz Lis,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.00,0.00,0.00,0.76,0.62,0.76
587,Willy Caballero,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.00,0.00,0.00,0.76,0.62,0.76


In [322]:
updated_training_data = pd.concat([updated_training_data, gw4])
updated_training_data = updated_training_data.reset_index()
updated_training_data = updated_training_data.drop('index', axis=1)
updated_training_data

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,2,1,Arsenal,Fulham,False,2021,90,6,0,...,4.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
1,Alexandre Lacazette,4,1,Arsenal,Fulham,False,2021,86,7,0,...,48.0,0,3,0,0.4,0.2,0.4,1.41,0.96,1.29
2,Bernd Leno,1,1,Arsenal,Fulham,False,2021,90,7,0,...,0.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
3,Bukayo Saka,3,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
4,Calum Chambers,2,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51543,Will Smallbone,3,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51544,Tino Livramento,2,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51545,Mateusz Lis,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51546,Willy Caballero,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76


In [332]:
#Make sure all team names are the same in entire df
updated_training_data.team.unique()

array(['Arsenal', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Leeds', 'Leicester City', 'Liverpool', 'Newcastle Utd',
       'Sheffield Utd', 'Southampton', 'Tottenham', 'West Brom',
       'West Ham', 'Wolves', 'Aston Villa', 'Burnley', 'Manchester City',
       'Manchester Utd', 'Brentford', 'Norwich', 'Watford', 'Bournemouth',
       'Nottingham Forest'], dtype=object)

In [334]:
#Make sure all opponent_team names are the same in entire df
updated_training_data.opponent_team.unique()

array(['Fulham', 'Chelsea', 'Brighton', 'Southampton', 'Tottenham',
       'Arsenal', 'Liverpool', 'West Brom', 'Leeds', 'West Ham', 'Wolves',
       'Crystal Palace', 'Everton', 'Leicester City', 'Newcastle Utd',
       'Sheffield Utd', 'Manchester Utd', 'Burnley', 'Aston Villa',
       'Manchester City', 'Brentford', 'Watford', 'Norwich',
       'Nottingham Forest', 'Bournemouth'], dtype=object)

In [333]:
#Save updated training data 
updated_training_data.to_csv(path/'training_data_updated.csv', index=False)
updated_training_data

,player,position,gw,team,opponent_team,was_home,season,minutes,total_points,assists,...,threat,yellow_cards,team_a_score,team_h_score,xg,xa,npxg,team_xg,team_xa,team_npxg
0,Ainsley Maitland-Niles,2,1,Arsenal,Fulham,False,2021,90,6,0,...,4.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
1,Alexandre Lacazette,4,1,Arsenal,Fulham,False,2021,86,7,0,...,48.0,0,3,0,0.4,0.2,0.4,1.41,0.96,1.29
2,Bernd Leno,1,1,Arsenal,Fulham,False,2021,90,7,0,...,0.0,0,3,0,0.0,0.0,0.0,1.41,0.96,1.29
3,Bukayo Saka,3,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
4,Calum Chambers,2,1,Arsenal,Fulham,False,2021,0,0,0,...,0.0,0,3,0,NaN,NaN,NaN,1.41,0.96,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51543,Will Smallbone,3,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51544,Tino Livramento,2,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51545,Mateusz Lis,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
51546,Willy Caballero,1,3,Southampton,Leicester City,False,2223,0,0,0,...,0.0,0,2,1,0.0,0.0,0.0,0.76,0.62,0.76
